In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
!unzip /content/drive/My\ Drive/Copy\ of\ test.zip -d test


In [ ]:
!unzip /content/drive/My\ Drive/Copy\ of\ train.zip -d train

In [ ]:
import numpy as np
import pandas as pd
import os
#import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from keras.layers import Convolution2D
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import load_model, Model, Sequential, save_model
from tensorflow.keras.layers import Input, Conv2D, Add, BatchNormalization, concatenate
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator


# from tensorflow.keras.datasets.mnist import load_data
# from tensorflow.keras.models import load_model, save_model,Model, Sequential
# from tensorflow.keras.layers import *
# from keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import KFold

Using TensorFlow backend.


In [ ]:
submission = pd.read_csv('/content/drive/My Drive/data_mining/sample_submission_yj.csv')

In [ ]:
train_files = os.listdir('train')

In [ ]:
test = []
for sub_id in submission['id']:
    data = np.load('test/'+'subset_'+sub_id+'.npy').astype('float32')
    test.append(data)

In [ ]:
train = []
for file in train_files:
    try:
        data = np.load('train/'+file).astype('float32')
        train.append(data)
    except:
        continue

In [ ]:
train = np.array(train)
test = np.array(test)

In [ ]:
x_train = train[:,:,:,:10]
y_train = train[:,:,:,14]
test = test[:,:,:,:10]

del train

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.025, random_state=7777)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((74436, 40, 40, 10), (74436, 40, 40), (1909, 40, 40, 10), (1909, 40, 40))

In [ ]:
y_train_ = y_train.reshape(-1,y_train.shape[1]*y_train.shape[2])

x_train = np.delete(x_train, np.where(y_train_<0)[0], axis=0)
y_train = np.delete(y_train, np.where(y_train_<0)[0], axis=0)
y_train = y_train.reshape(-1, x_train.shape[1], x_train.shape[2],1)
y_test = y_test.reshape(-1, y_test.shape[1], y_test.shape[2],1)

y_train_ = np.delete(y_train_, np.where(y_train_<0)[0], axis=0)

x_train.shape, y_train.shape

((74055, 40, 40, 10), (74055, 40, 40, 1))

In [ ]:
x_train = x_train[np.sum(y_train_, axis=1) >= 50]
y_train = y_train[np.sum(y_train_, axis=1) >= 50]

x_train.shape, y_train.shape

((30642, 40, 40, 10), (30642, 40, 40, 1))

In [ ]:
def mae_over_fscore(y_true, y_pred):
    
    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    IsNotMissing = y_true >= 0
    
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    f_score = f1_score(y_true, y_pred) 
    
    return mae / (f_score + 1e-07) 

In [ ]:
def mae(y_true, y_pred):    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def score(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [ ]:
def create_model():
    inputs=Input(x_train.shape[1:])
    
    bn=BatchNormalization()(inputs)
    conv0=Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn=BatchNormalization()(conv0)
    conv=Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([conv0, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    conv=Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([concat, conv], axis=3)
        
    for i in range(5):
        bn=BatchNormalization()(concat)
        conv=Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat=concatenate([concat, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    outputs=Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model=Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
def create_vgg16():   
    model = Sequential()
    model.add(Input((40,40,10)))
    model.add(BatchNormalization())
    for i in range(2):
        model.add(Conv2D(64, kernel_size=3, padding = 'same', activation = 'relu', strides = 1))


    model.add(BatchNormalization())
    for i in range(2):
        model.add(Conv2D(128, kernel_size=3, padding = 'same', activation = 'relu', strides = 1))

    model.add(BatchNormalization())
    for i in range(3):
        model.add(Conv2D(256, kernel_size=3, padding = 'same', activation = 'relu', strides = 1))

    model.add(BatchNormalization())
    for i in range(5):
        model.add(Conv2D(512, kernel_size=3, padding = 'same', activation = 'relu', strides = 1))

    model.add(BatchNormalization())
    model.add(Conv2D(1, kernel_size=1, padding = 'same', activation = 'relu', strides = 1))
    
    return model

In [ ]:
def inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32):
   
    x_1 = Conv2D(o_1, 1, padding='same',activation='relu')(x)
    
    x_2 = Conv2D(r_3, 1, padding='same',activation='relu')(x)
    x_2 = Conv2D(o_3, 3, padding='same',activation='relu')(x_2)
    
    x_3 = Conv2D(r_5, 1, padding='same',activation='relu')(x)
    x_3 = Conv2D(o_5, 5, padding='same',activation='relu')(x_3)
    
    x_4 = MaxPooling2D(pool_size=(3, 3), strides=1, padding='same')(x)
    x_4 = Conv2D(pool, 1, padding='same',activation='relu')(x_4)
    
    return concatenate([x_1, x_2, x_3, x_4])

def create_inception():
    input_ = Input((40, 40, 10))
    x = Conv2D(64, 7, strides=1, padding='same', activation='relu')(input_)
    x = BatchNormalization()(x)
    x = Conv2D(64, 1, strides=1)(x)
    x = Conv2D(192, 3, strides=1, padding='same',activation='relu')(x)
    x = BatchNormalization()(x)
    
    for i in range(5):
        x = inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32)

    x = BatchNormalization()(x);
    output = Conv2D(1, 1, strides=1, padding='same', activation='relu')(x)

    googlenet = Model(input_, output)
    return googlenet

In [ ]:
def create_dense():
    inputs = Input((40,40,10))
    bn=BatchNormalization()(inputs)
    conv=Conv2D(64, kernel_size=7, strides=1, padding='same', activation='relu')(bn)

    for i in range(6):
        bn=BatchNormalization()(conv)
        x=Conv2D(128, kernel_size=1, padding='same', activation='relu')(bn)
        x=BatchNormalization()(x)
        x=Conv2D(32, kernel_size=3, padding='same',activation='relu')(x)
        conv = concatenate([conv, x], axis=3)
        
    bn=BatchNormalization()(conv)
    outputs=Conv2D(1,kernel_size=1, padding='same',activation='relu')(bn)
    return Model(inputs=inputs, outputs=outputs)

In [ ]:
def train_model(x_data, y_data, k, s):
    k_fold = KFold(n_splits=k, shuffle=True, random_state=7777)
    model_number = 0
    for train_idx, val_idx in k_fold.split(x_data):
        if model_number == s:
            x_train, y_train = x_data[train_idx], y_data[train_idx]
            x_val, y_val = x_data[val_idx], y_data[val_idx]

            # model = create_model()
            # model = create_vgg16()
            # model = create_inception()
            model = create_dense()
            model.compile(loss='mae', optimizer='adam', metrics=[score, fscore_keras])

            callbacks_list = [
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    patience=3,
                    factor=0.8
                ),

                tf.keras.callbacks.ModelCheckpoint(
                    filepath = 'models/model'+str(model_number)+'.h5',
                    monitor='val_score',
                    save_best_only=True
                )
            ]

            model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=callbacks_list)
        
        model_number+=1

In [ ]:
k = 5
models = []

train_model(x_train, y_train, k=k, s=4)

model = load_model('models/model_inception.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
models.append(model)

In [1]:
models = []
model1 = load_model('/content/drive/My Drive/data_mining/my_model/inception_v1_4.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
models.append(model1)
model2 = load_model('/content/drive/My Drive/data_mining/my_model/model_vgg.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
models.append(model2)
model3 = load_model('/content/drive/My Drive/data_mining/my_model/resnet_v1_0.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
models.append(model3)
model4 = load_model('/content/drive/My Drive/data_mining/my_model/resnet_v1_1.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
models.append(model4)
model5 = load_model('/content/drive/My Drive/data_mining/my_model/resnet_v1_2.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
models.append(model5)
model6 = load_model('/content/drive/My Drive/data_mining/my_model/resnet_v1_3.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
models.append(model6)
model7 = load_model('/content/drive/My Drive/data_mining/my_model/resnet_v1_4.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
models.append(model7)
preds = []
for model in models:
    preds.append(model.predict(x_test))
    
pred = sum(preds)/len(preds)
print(mae_over_fscore(y_test, pred))

NameError: ignored

In [ ]:
preds = []
for model in models:
    preds.append(model.predict(test))

pred = sum(preds)/len(preds)

In [ ]:
submission.iloc[:,1:] = pred.reshape(-1,1600)
submission

,id,px_1,px_2,px_3,px_4,px_5,px_6,px_7,px_8,px_9,px_10,px_11,px_12,px_13,px_14,px_15,px_16,px_17,px_18,px_19,px_20,px_21,px_22,px_23,px_24,px_25,px_26,px_27,px_28,px_29,px_30,px_31,px_32,px_33,px_34,px_35,px_36,px_37,px_38,px_39,...,px_1561,px_1562,px_1563,px_1564,px_1565,px_1566,px_1567,px_1568,px_1569,px_1570,px_1571,px_1572,px_1573,px_1574,px_1575,px_1576,px_1577,px_1578,px_1579,px_1580,px_1581,px_1582,px_1583,px_1584,px_1585,px_1586,px_1587,px_1588,px_1589,px_1590,px_1591,px_1592,px_1593,px_1594,px_1595,px_1596,px_1597,px_1598,px_1599,px_1600
0,029858_01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.067286,0.96256,0.014102,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,029858_02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.009607,0.283757,0.0,0.000000,0.041569,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.355397,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.468787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,029858_03,0.000000,0.005924,0.060267,0.291189,0.024914,0.000000,0.000000,0.0,0.0,0.0,0.018262,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.003581,0.059921,0.833282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,029858_05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.590125,1.543256,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003342,0.584539,2.204974,8.846297,1.574994,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,029858_07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,1.105329,1.207428,1.186699,1.743607,2.866126,3.429738,2.298806,0.882816,0.800451,1.789101,3.215748,6.009251,5.395774,4.499981,8.129619,12.636506,19.637285,27.211067,24.568033,23.531523,13.84586,8.779487,13.588128,20.692343,7.534243,1.311942,0.226168,0.129106,0.357201,0.979169,2.162693,2.728006,1.822258,0.827659,0.966549,2.446307,5.048596,3.883776,2.315774,1.43806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
submission.to_csv('/content/drive/My Drive/data_mining/submission_not_incep.csv', index=False)